In [ ]:

%run includes/3-network-setup.ipynb

# Networks
Summary and detail reports on networks and subnetworks deployed across the GCP estate.

In [ ]:

# get networks
network_queries = [
    f"""
    SELECT name,
    '{project}' as project,
    mtu,
    networkFirewallPolicyEnforcementOrder as fwPolicyEnforcementOrder,
    JSON_ARRAY_LENGTH(peerings) as num_peerings,
    peerings,
    JSON_EXTRACT(routingConfig, '$.routingMode') as routingMode,
    JSON_ARRAY_LENGTH(subnetworks) as num_subnets
    FROM google.compute.networks WHERE project = '{project}'
    AND name IS NOT NULL
    """
    for project in compute_enabled_projects
]
networks_df = run_stackql_queries(network_queries)
networked_projects = list(networks_df['project'].unique())

# get interconnects
interconnect_queries = [
    f"""
    SELECT *
    FROM google.compute.interconnects WHERE project = '{project}'
    AND name IS NOT NULL
    """
    for project in networked_projects
]
interconnects_df = run_stackql_queries(interconnect_queries)

# get vpns
vpn_queries = [
    f"""
    SELECT
    name,
    '{project}' as project, 
    SPLIT_PART(network, '/', 9) as network,
    SPLIT_PART(region, '/', 8) as region,
    JSON_ARRAY(JSON_EXTRACT(vpnInterfaces, '$[0].ipAddress'), JSON_EXTRACT(vpnInterfaces, '$[1].ipAddress')) as ip_addresses
    FROM google.compute.vpn_gateways WHERE project = '{project}'
    """
    for project in networked_projects
]
vpns_df = run_stackql_queries(vpn_queries)

external_vpn_gateways_queries = [
    f"""
    select 
    name
    ,description
    ,JSON_ARRAY_LENGTH(interfaces) as num_interfaces
    ,JSON_ARRAY(JSON_EXTRACT(interfaces, '$[0].ipAddress'), JSON_EXTRACT(interfaces, '$[1].ipAddress'), JSON_EXTRACT(interfaces, '$[2].ipAddress'), JSON_EXTRACT(interfaces, '$[3].ipAddress')) as ip_addresses
    ,redundancyType
    ,labels 
    from google.compute.external_vpn_gateways where project = '{project}'
    and name is not null
    """
    for project in networked_projects
]
external_vpn_gateways_df = run_stackql_queries(external_vpn_gateways_queries)

# get firewalls
firewall_queries = [
    f"""
    select 
    name
    ,sourceRanges
    ,destinationRanges
    ,sourceServiceAccounts
    ,sourceTags
    ,targetServiceAccounts
    ,targetTags
    ,denied
    ,direction
    ,SPLIT_PART(network, '/', 10) as network
    ,allowed
    ,priority
    ,disabled
    ,logConfig
    ,description
    from google.compute.firewalls where project = '{project}' and name is not null
    """
    for project in networked_projects
]
firewalls_df = run_stackql_queries(firewall_queries)

# forwarding rules
forwarding_rule_queries = [
    f"""
    select * from google.compute.forwarding_rules where project = '{project}'
    """
    for project in networked_projects
]
forwarding_rule_df = run_stackql_queries(forwarding_rule_queries)
forwarding_rule_df

## Network Summary

In [ ]:

# show network summary data
show_network_summary_cards(networks_df, len(interconnects_df), len(vpns_df), len(external_vpn_gateways_df), len(firewalls_df))

## Network Peerings

In [ ]:

# get network peerings
networks_subset_df = networks_df[networks_df['peerings'] != 'null'][['name', 'peerings']].copy()
network_peerings_df = (explode_json_list_col(networks_subset_df, 'peerings', 'peering')
                       .assign(peering_network=lambda x: x['peering_network'].str.split('/').str[-1])
                      )

### Network Peering Graph

In [ ]:

# graph peerings
graph_network_peerings(network_peerings_df)

### Network Peering Details

In [ ]:

# show network peering details
network_peerings_df

## Subnetworks

In [ ]:

# get and show subnetworks
queries = [
    f"""
    SELECT 
    name,
    SPLIT_PART(region, '/', 9) as region,
    SPLIT_PART(network, '/', 10) as network,
    stackType,
    ipCidrRange,
    secondaryIpRanges,
    gatewayAddress,
    ipv6CidrRange,
    purpose,
    enableFlowLogs,
    privateIpGoogleAccess
    FROM google.compute.subnetworks WHERE project = '{project}'
    """
    for project in networked_projects
]

subnetworks_df = run_stackql_queries(queries)
subnetworks_df

## Addresses

In [ ]:

# get addresses
address_queries = [
    f"""
    SELECT 
    id,
    name,
    '{project}' as project, 
    SPLIT_PART(network, '/', 9) as network,
    SPLIT_PART(subnetwork, '/', 11) as subnetwork,
    addressType,
    address,
    ipVersion,
    ipv6EndpointType,
    purpose,
    labels,
    status,
    users,
    networkTier
    FROM google.compute.addresses WHERE project = '{project}'
    """
    for project in networked_projects
]
addresses_df = run_stackql_queries(address_queries)

global_address_queries = [
    f"""
    select * from google.compute.global_addresses where project = '{project}' and name is not null
    """
    for project in networked_projects
]
global_address_df = run_stackql_queries(global_address_queries)

### Address Details

In [ ]:

# show addresses
addresses_df

### Global Address Details

In [ ]:

# show global addresses
global_address_df

## Hybrid Connectivity

In [ ]:

# vpn tunnels
vpn_tunnels_queries = [
    f"""
    select * from google.compute.vpn_tunnels where project = '{project}'
    """
    for project in networked_projects
]
vpn_tunnels_df = run_stackql_queries(vpn_tunnels_queries)

### VPN Gateways

In [ ]:

# show vpn gateways
vpns_df

### External VPN Gateways

In [ ]:

# show external vpn gateways
external_vpn_gateways_df

### VPN Tunnels

In [ ]:

# show vpn tunnels
vpn_tunnels_df

## Firewalls

In [ ]:

# show firewalls
firewalls_df

## Routers and Routes

In [ ]:

# routers
routers_queries = [
    f"""
    select 
    name
    ,description
    ,SPLIT_PART(region, '/', 9) as region	
    ,SPLIT_PART(network, '/', 10) as network
    ,nats
    ,bgp
    ,bgpPeers
    ,JSON_ARRAY_LENGTH(interfaces) as num_interfaces
    ,interfaces
    ,encryptedInterconnectRouter
    ,md5AuthenticationKeys		 	
    from google.compute.routers where project = '{project}'
    """
    for project in networked_projects
]
routers_df = run_stackql_queries(routers_queries)

routes_queries = [
    f"""
    select 
    name
    ,description
    ,'{project}' as project
    ,SPLIT_PART(network, '/', 10) as network
    ,SPLIT_PART(nextHopInstance, '/', 11) as nextHopInstance
    ,tags
    ,nextHopIlb	
    ,destRange	
    ,nextHopHub	
    ,SPLIT_PART(nextHopGateway, '/', 10) as nextHopGateway
    ,routeStatus
    ,priority
    ,nextHopVpnTunnel
    ,routeType
    ,nextHopIp
    ,warnings
    ,creationTimestamp
    ,asPaths
    ,nextHopPeering
    ,SPLIT_PART(nextHopNetwork, '/', 10) as nextHopNetwork
    from google.compute.routes where project = '{project}'
    """
    for project in networked_projects
]
routes_df = run_stackql_queries(routes_queries)

### Routers

In [ ]:

# show routers
routers_df

### Routes

In [ ]:

# show routes
routes_df